# Case study: asset utilisation (with timezones)

This case study illustrates the use of the staircase package for analysing asset utilsation.  In this example we have a group of 3 identical assets, X, Y, and Z.  These assets are parallel machines, such as bays in a carwash.  Each asset can either be in one of two states - on or off (or alternatively working or not working).

The data used is this case study is synthetic and fictional.  Both data and the notebook for this tutorial can be obtained from the [github site](https://github.com/venaturum/staircase/tree/master/docs/examples).

In [ ]:
import pandas as pd
import staircase as sc
import matplotlib.pyplot as plt
import pytz
from matplotlib import rcParams 

In [ ]:
timezone = pytz.timezone('Australia/Sydney')
rcParams['timezone'] = 'Australia/Sydney'
tz_timestamp = lambda t: pd.Timestamp(t, tz=timezone)

We begin by importing the asset data into a pandas.DataFrame instance. Each row corresponds to a period of time that an asset is being used.  The first column identifies the asset, while the second and third columns give the start and end times of the period of use respectively.

In [ ]:
data = pd.read_csv(r"./data/asset_use.csv", parse_dates=['start', 'end'], dayfirst=True)
data.start = data.start.dt.tz_localize('Australia/Sydney')
data.end = data.end.dt.tz_localize('Australia/Sydney')
data

For the analysis we would like a Stairs object for each asset.  Each Stairs object will represent a step function which takes a value of zero, when the asset is not in use, and a value of one when the asset is in use.  We define a function called "make_stairs" which, given a dataframe, creates a Stairs instance based on columns called "start" and "end".  We can then use this function, in conjunction with a groupby-apply, to create a pandas.Series indexed by asset name, with Stairs values.

In [ ]:
make_stairs = lambda df: sc.Stairs(0, df.start, df.end)

asset_use = data.groupby(['asset']).apply(make_stairs)

We can access an individual Stairs object with the corresponding asset name.  For example, to plot the step function corresponding to asset Z:

In [ ]:
asset_use['Z'].plot()

A wall of blue!  Perhaps our asset is oscillating between not used and used (0 amd 1 respectively) so often that plotting an entire year is not helpful.  Let's confirm this conjecture by capturing the axes which is being plotted onto, and "zooming in" on the first day of 2020:

In [ ]:
ax = asset_use['Z'].plot()
ax.xaxis_date('Australia/Sydney')
ax.set_xlim(tz_timestamp('2020-1-1'), tz_timestamp('2020-1-2'))

Because these assets belong to a group, we are interested in their combined utilisation, i.e. the addition of the three step functions.  This can be achieved by simply summing up the Stairs objects:

In [ ]:
combined_asset_use = asset_use['X'] + asset_use['Y'] + asset_use['Z']

Note that we can also achieve the same result by leveraging the panda.Series.sum method.  That method leverages the addition operator defined by the Stairs class to perform the calculation.

In [ ]:
combined_asset_use = asset_use.sum()

Using the integrate method we can see that the three assets together worked for a total of 13,176.7 hours in the year 2020.

In [ ]:
yr2020 = (tz_timestamp('2020'),tz_timestamp('2021'))
combined_asset_use.integrate(*yr2020)

Given there are 3 assets, and 8784 hours in the year, there are a total of 26,352 hours that the assets could have worked - approximately double the hours used.  It is not surprising then that calculating the average group utilisation for the year 2020 is approximately 1.5: 

In [ ]:
combined_asset_use.mean(*yr2020)

Now we will look to discover how often exactly none of the assets were being used.  If we compare our combined_asset_use Stairs instance to 0, then the result is a Stairs instance.  This object represents a binary (or boolean) valued step function, which takes value 1 whenever there are 0 assets being used.

In [ ]:
combined_asset_use == 0

The question, of how often none of the assets are being used, can be answered with the help of the mean method:

In [ ]:
(combined_asset_use == 0).mean(*yr2020)

So there are no assets being used at all, during 2020, approximately 12.3% of the time.  If we wish to further this idea by extending to 1, 2, or 3 assets being used, then we can do so with a list comprehension to store each result.  We wrap this in a pandas.Series for convenience:

In [ ]:
combined_utilisation = pd.Series([(combined_asset_use == i).mean(*yr2020) for i in range(4)])
combined_utilisation

We can see from this result that all assets are being used approximately 10.1% of the time.  Since the values are fractions of a whole, if we sum the values in the series we should get a value of 1.  Let's check:

In [ ]:
combined_utilisation.sum()

In [ ]:
combined_utilisation = combined_asset_use.hist()

In [ ]:
combined_utilisation

We can also use the pandas.Series plotting methods to get a quick visual of this utilisation breakdown:

In [ ]:
combined_utilisation.plot.bar()

Using various methods from the Stairs class we can answer questions such as "How often is X working while Y is not?": (25.4%)

In [ ]:
(asset_use["X"] > asset_use["Y"]).mean(*yr2020)

"How many assets were being used at 9:40am on the 5th of September?: (2)

In [ ]:
combined_asset_use(tz_timestamp('2020-9-5 9:40'))

"Which 2 assets were being used at this time?": (X and Y)

In [ ]:
asset_use.apply(lambda s: s(tz_timestamp('2020-9-5 9:40')))

If we'd prefer to work with "idleness", instead of "in use", we can create a Stairs object to capture this.  The idea is to start with a step function with value 3 everywhere (since we have 3 assets) and subtract the step function for "in use":

In [ ]:
combined_assets_idle = sc.Stairs(3)-combined_asset_use

The number of assets idle at 9:40am on the 5th of September should be 1 right?  Let's check:

In [ ]:
combined_assets_idle(tz_timestamp('2020-9-5 9:40'))